## Module 1.3

In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json',  'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields=['course']
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
# default boost of 1.0
boost = {
    'question': 3.0,
    'section': 0.5
}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'course': 'data-engineering-zoomcamp'}
    
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Module 1.4

In [12]:
from google import genai
import os

GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

client = genai.Client(api_key=GEMINI_API_KEY)

In [13]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    # config=types.GenerateContentConfig(
        # system_instruction="You are a cat. Your name is Neko."),
    contents=q
)

In [14]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="Whether or not you can enroll in a course that has already started depends on a few factors:\n\n*   **The institution's policy:** Some schools or organizations have strict deadlines for enrollment, while others are more flexible.\n*   **The type of course:** Online, self-paced courses often allow enrollment at any time. In-person courses with a fixed schedule might be less flexible.\n*   **The length of the course:** If the course has just started, you might still be able to join. If it's already halfway through, it's less likely.\n*   **The instructor's discretion:** In some cases, the instructor might be willing to let you join if you can catch up on the missed material.\n\n**Here's what I recommend you do:**\n\n1.  **Che

In [15]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""

In [17]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

In [22]:
prompt = prompt_template.format(question=q, context=context).strip()

In [23]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    # config=types.GenerateContentConfig(
        # system_instruction="You are a cat. Your name is Neko."),
    contents=prompt
)

In [24]:
response.text

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.\n"